In [1]:
import pandas as pd

In [2]:
# read db export
df = pd.read_csv("exp_db.csv").dropna(how="all")
# select where to start and stop generating
df = df.loc[df["Short name"].apply(lambda x: 1483 < int(x[2:]) < 1525)]
df

,Short name,Date,Data set ID,Split ID,Estimator,Features,Tags
385,JG1484,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,Logistic Regression,FP,NaN
386,JG1485,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,Logistic Regression,RDKit,NaN
387,JG1486,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,XGB,FP,NaN
388,JG1487,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,XGB,"FP, RDKit",NaN
389,JG1488,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,FFN,OHE,NaN
390,JG1489,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,D-MPNN,CGR,NaN
391,JG1490,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,D-MPNN,"CGR, RDKit",NaN
392,JG1491,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,AttentiveFP,CGR,NaN
393,JG1492,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_1D_split,GraphSAGE,CGR,NaN
394,JG1493,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_2D_split,Logistic Regression,FP,NaN


In [3]:
def get_features(s):
    if s == "CGR":
        return "None"
    s = s.replace(",", "")
    s = s.replace("CGR", "")
    s = s.strip()
    return s
        

In [8]:
for i, row in df.iterrows():
    model = row["Estimator"].replace("-", "").replace("Logistic Regression", "LogReg")
    global_features = get_features(row["Features"])
    exp_id = row["Short name"]
    split = row["Split ID"]
    tags = "_".join([row["Split ID"].split("_")[-3], row["Split ID"].split("_")[-1]])  # for "0D_80" style
    tags = row["Split ID"].split("_")[-2] # for "1D" style
    print(f"nq /home/julian/miniconda3/envs/synferm-predictions/bin/python /home/julian/PycharmProjects/synferm-predictions/run.py train --config config/config_{model}.yaml --data-path data/synferm_dataset_2023-12-20_39486records.csv --split-indices data/splits/{split}/ --smiles-columns reaction_smiles_atom_mapped --label-columns binary_A binary_B binary_C --global-features {global_features} --global-features-file None --task multilabel --run-test --hparam-optimization --hparam-config-path config/hparam_bounds_{model}.yaml --hparam-n-iter 20 --experiment-id {exp_id} --tags {tags}")
    print()


nq /home/julian/miniconda3/envs/synferm-predictions/bin/python /home/julian/PycharmProjects/synferm-predictions/run.py train --config config/config_LogReg.yaml --data-path data/synferm_dataset_2023-12-20_39486records.csv --split-indices data/splits/synferm_dataset_2023-12-20_1D_split/ --smiles-columns reaction_smiles_atom_mapped --label-columns binary_A binary_B binary_C --global-features FP --global-features-file None --task multilabel --run-test --hparam-optimization --hparam-config-path config/hparam_bounds_LogReg.yaml --hparam-n-iter 20 --experiment-id JG1484 --tags 1D

nq /home/julian/miniconda3/envs/synferm-predictions/bin/python /home/julian/PycharmProjects/synferm-predictions/run.py train --config config/config_LogReg.yaml --data-path data/synferm_dataset_2023-12-20_39486records.csv --split-indices data/splits/synferm_dataset_2023-12-20_1D_split/ --smiles-columns reaction_smiles_atom_mapped --label-columns binary_A binary_B binary_C --global-features RDKit --global-features-fil